# `specfile_example()` - Output scan(s) to a SPEC data file.

One of the common concerns is how to access data from bluesky's database.  The standard way is to replay the document stream from each of the scans through a bluesky callback that writes the data to the desired file format.  Here, we write data to the SPEC file format.

First, we must load the libraries we'll need.

In [1]:
from databroker import Broker
from apstools.examples import specfile_example

Next, we create an instance of the *Broker()* using our mongodb database. (For this to work, we know we already have a file located in `~/.config/databroker/mongodb_config.yml` that describes the databroker configuration for mongodb.)

In [2]:
db = Broker.named("mongodb_config")

# get the most recent scan, by steps

The databroker instance, `db`, provides access to its scans by several means.  One way is to consider `db` as a list and retreive the last item from the list.  This will return a *header* to the scan.  The *header* is the common reference to be used.  As is the common term, we will call it `h` and print its *start* document.

For this first example, we'll work through the steps one by one.

In [3]:
h = db[-1]
h.start

INFO:databroker._core:Interpreting key = -1 as an integer


{'time': 1550617943.5272377,
 'uid': '957d83c1-9f11-47df-936b-ce14a396cd76',
 'plan_name': 'TuneAxis.tune',
 'tune_parameters': {'num': 30,
  'width': 2.5,
  'initial_position': -1.5,
  'peak_choice': 'cen',
  'x_axis': 'm1',
  'y_axis': 'spvoigt'},
 'hints': {'dimensions': [[['m1'], 'primary']]},
 'scan_id': 5,
 'plan_type': 'generator',
 'md': {'activity': 'TuneAxis development and testing',
  'peak_model': 'pseudo Voigt',
  'peak_scale': 100000.0,
  'peak_center': -1.3940973681450914,
  'peak_sigma': 0.025534621641250733,
  'peak_eta': 0.2993015167776747,
  'peak_bkg': 0.0045580721055284755},
 'tune_md': {'width': 2.5,
  'initial_position': -1.5,
  'time_iso8601': '2019-02-19 17:12:23.523994'}}

The databroker provides a simple table view of this scan (header):

In [4]:
h.table()

,time,spvoigt,m1,m1_user_setpoint
seq_num,,,,
1,2019-02-19 17:12:25.014852762,466.418222,-2.75,-2.750000
2,2019-02-19 17:12:25.290538549,467.980011,-2.66,-2.663793
3,2019-02-19 17:12:25.583233356,469.676953,-2.58,-2.577586
4,2019-02-19 17:12:25.886490583,472.047289,-2.49,-2.491379
5,2019-02-19 17:12:26.187965870,474.704056,-2.41,-2.405172
6,2019-02-19 17:12:26.487434149,478.553311,-2.32,-2.318966
7,2019-02-19 17:12:26.789104462,483.710237,-2.23,-2.232759
8,2019-02-19 17:12:27.089837313,489.921848,-2.15,-2.146552
9,2019-02-19 17:12:27.390726089,499.752081,-2.06,-2.060345


Let's write it as a SPEC data file (namely: `spec1.dat`):

In [5]:
specfile_example(h, filename="spec1.dat")

INFO:apstools.filewriters:wrote header to SPEC file: spec1.dat
INFO:apstools.filewriters:wrote scan 5 to SPEC file: spec1.dat
INFO:apstools.examples:
#S 5  TuneAxis.tune()
#D Tue Feb 19 17:12:23 2019
#C Tue Feb 19 17:12:23 2019.  plan_type = generator
#C Tue Feb 19 17:12:23 2019.  uid = 957d83c1-9f11-47df-936b-ce14a396cd76
#MD md = {'activity': 'TuneAxis development and testing', 'peak_model': 'pseudo Voigt', 'peak_scale': 100000.0, 'peak_center': -1.3940973681450914, 'peak_sigma': 0.025534621641250733, 'peak_eta': 0.2993015167776747, 'peak_bkg': 0.0045580721055284755}
#MD tune_md = {'width': 2.5, 'initial_position': -1.5, 'time_iso8601': '2019-02-19 17:12:23.523994'}
#MD tune_parameters = {'num': 30, 'width': 2.5, 'initial_position': -1.5, 'peak_choice': 'cen', 'x_axis': 'm1', 'y_axis': 'spvoigt'}
#N 30
#L Epoch_float  m1  m1_user_setpoint  spvoigt  Epoch
1.4876151084899902 -2.75 -2.75 466.4182216464095 1
1.763300895690918 -2.66 -2.663793103448276 467.98001092446935 2
2.05599570274353

Let's view that file from disk storage:

In [6]:
!cat spec1.dat

#F spec1.dat
#E 1550619537
#D Tue Feb 19 17:38:57 2019
#C BlueSky  user = mintadmin  host = mint-vm

#S 5  TuneAxis.tune()
#D Tue Feb 19 17:12:23 2019
#C Tue Feb 19 17:12:23 2019.  plan_type = generator
#C Tue Feb 19 17:12:23 2019.  uid = 957d83c1-9f11-47df-936b-ce14a396cd76
#MD md = {'activity': 'TuneAxis development and testing', 'peak_model': 'pseudo Voigt', 'peak_scale': 100000.0, 'peak_center': -1.3940973681450914, 'peak_sigma': 0.025534621641250733, 'peak_eta': 0.2993015167776747, 'peak_bkg': 0.0045580721055284755}
#MD tune_md = {'width': 2.5, 'initial_position': -1.5, 'time_iso8601': '2019-02-19 17:12:23.523994'}
#MD tune_parameters = {'num': 30, 'width': 2.5, 'initial_position': -1.5, 'peak_choice': 'cen', 'x_axis': 'm1', 'y_axis': 'spvoigt'}
#N 30
#L Epoch_float  m1  m1_user_setpoint  spvoigt  Epoch
1.4876151084899902 -2.75 -2.75 466.4182216464095 1
1.763300895690918 -2.66 -2.663793103448276 467.98001092446935 2
2.0559957027435303 -2.58 -2.5775862068965516 469.67695329594545 2

We see that the output of the `specfile_example()` command includes the content of the SPEC file.  For the remaining examples, we'll skip this additional step to view the SPEC file contents from disk.

# a range of recent scans

We can continue to consider the `db` object as a list and use list slicing to access a range of recent scans.

In [7]:
specfile_example(db[-6:][::-3], filename="spec2.dat")

INFO:databroker._core:Interpreting key = slice(-6, None, None) as a slice
INFO:apstools.filewriters:wrote header to SPEC file: spec2.dat
INFO:apstools.filewriters:wrote scan 1 to SPEC file: spec2.dat
INFO:apstools.examples:
#S 1  scan(detectors=['scaler'], num=15, args=['m1', -5, 5], per_step=None)
#D Tue Feb 19 17:11:20 2019
#C Tue Feb 19 17:11:20 2019.  plan_type = generator
#C Tue Feb 19 17:11:20 2019.  uid = 225eef4b-4a56-427b-b507-e959b216a9f4
#MD motors = ['m1']
#MD num_intervals = 14
#MD num_points = 15
#MD plan_pattern = inner_product
#MD plan_pattern_args = {'num': 15, 'args': ["EpicsMotor(prefix='vm7:m1', name='m1', settle_time=0.0, timeout=None, read_attrs=['user_readback', 'user_setpoint'], configuration_attrs=['user_offset', 'user_offset_dir', 'velocity', 'acceleration', 'motor_egu'])", -5, 5]}
#MD plan_pattern_module = bluesky.plan_patterns
#N 15
#L m1  I0  m1_user_setpoint  scaler_time  scint  Epoch_float  Epoch  clock
-5.0 2.0 -5.0 0.5 3.0 5.246389150619507 5 5000000.0


# a specific scan

The `db` object allows us to access scans by UUID (or any shorter version that remains unique in the database).

In [8]:
specfile_example(db["37c188c0"], filename="spec3.dat")

INFO:databroker._core:Interpreting key = 37c188c0 as a str
INFO:apstools.filewriters:wrote header to SPEC file: spec3.dat
INFO:apstools.filewriters:wrote scan 3 to SPEC file: spec3.dat
INFO:apstools.examples:
#S 3  TuneAxis.multi_pass_tune()
#D Tue Feb 19 17:12:19 2019
#C Tue Feb 19 17:12:19 2019.  plan_type = generator
#C Tue Feb 19 17:12:19 2019.  uid = 37c188c0-4f24-4e9b-b8ab-d610dc8797c5
#MD md = {'activity': 'TuneAxis development and testing', 'peak_model': 'pseudo Voigt', 'peak_scale': 100000.0, 'peak_center': -1.3940973681450914, 'peak_sigma': 0.025534621641250733, 'peak_eta': 0.2993015167776747, 'peak_bkg': 0.0045580721055284755}
#MD pass = 3
#MD pass_max = 6
#MD tune_md = {'width': 0.15625, 'initial_position': -1.3900000000000001, 'time_iso8601': '2019-02-19 17:12:19.684373'}
#MD tune_parameters = {'num': 10, 'width': 0.15625, 'initial_position': -1.3900000000000001, 'peak_choice': 'cen', 'x_axis': 'm1', 'y_axis': 'spvoigt'}
#N 10
#L Epoch_float  m1  m1_user_setpoint  spvoigt 

# a list of specific scans, by UID

Suppose we have a list of scans where we know the UID of each one, we can build a list of headers and write a SPEC data file with that list.  Here, we have such a list of tuning scans.

In [9]:
hh = [db[uid] for uid in "957d83c c354fe37-e39f".split()]
specfile_example(hh, filename="spec_tunes.dat")

INFO:databroker._core:Interpreting key = 957d83c as a str
INFO:databroker._core:Interpreting key = c354fe37-e39f as a str
INFO:apstools.filewriters:wrote header to SPEC file: spec_tunes.dat
INFO:apstools.filewriters:wrote scan 5 to SPEC file: spec_tunes.dat
INFO:apstools.examples:
#S 5  TuneAxis.tune()
#D Tue Feb 19 17:12:23 2019
#C Tue Feb 19 17:12:23 2019.  plan_type = generator
#C Tue Feb 19 17:12:23 2019.  uid = 957d83c1-9f11-47df-936b-ce14a396cd76
#MD md = {'activity': 'TuneAxis development and testing', 'peak_model': 'pseudo Voigt', 'peak_scale': 100000.0, 'peak_center': -1.3940973681450914, 'peak_sigma': 0.025534621641250733, 'peak_eta': 0.2993015167776747, 'peak_bkg': 0.0045580721055284755}
#MD tune_md = {'width': 2.5, 'initial_position': -1.5, 'time_iso8601': '2019-02-19 17:12:23.523994'}
#MD tune_parameters = {'num': 30, 'width': 2.5, 'initial_position': -1.5, 'peak_choice': 'cen', 'x_axis': 'm1', 'y_axis': 'spvoigt'}
#N 30
#L Epoch_float  m1  m1_user_setpoint  spvoigt  Epoch

## "TuneAxis.multi_pass_tune" plans within a range of dates

The `db` object allows for filtering arguments based on any keywords in the *start* document and also by time.  Here, we filter between certain dates and also by `plan name`.  The dates are specified in ISO8601 format and can include precision beyond a millisecond.  Also, we write to the default data file: `test_specdata.txt`.

In [10]:
hh = db(plan_name="TuneAxis.multi_pass_tune", since="2019-02-19 17:00", until="2019-02-19 17:11:30")
for h in hh:
    print(h.start["uid"][:8], h.start["tune_md"]["time_iso8601"])
specfile_example(hh)

INFO:apstools.filewriters:wrote header to SPEC file: test_specdata.txt
INFO:apstools.filewriters:wrote scan 3 to SPEC file: test_specdata.txt
INFO:apstools.examples:
#S 3  TuneAxis.multi_pass_tune()
#D Tue Feb 19 17:10:49 2019
#C Tue Feb 19 17:10:49 2019.  plan_type = generator
#C Tue Feb 19 17:10:49 2019.  uid = 30d80cb1-96b7-4453-9f35-b32de8286d5d
#MD md = {'activity': 'TuneAxis development and testing', 'peak_model': 'pseudo Voigt', 'peak_scale': 100000.0, 'peak_center': -1.1908101513195635, 'peak_sigma': 0.04845116177798686, 'peak_eta': 0.5791263537920676, 'peak_bkg': 0.009752099380040652}
#MD pass = 3
#MD pass_max = 6
#MD tune_md = {'width': 0.15625, 'initial_position': -1.19, 'time_iso8601': '2019-02-19 17:10:49.360310'}
#MD tune_parameters = {'num': 10, 'width': 0.15625, 'initial_position': -1.19, 'peak_choice': 'cen', 'x_axis': 'm1', 'y_axis': 'spvoigt'}
#N 10
#L Epoch_float  m1  m1_user_setpoint  spvoigt  Epoch
0.3052546977996826 -1.27 -1.268125 27817.58713004543 0
0.487897396

30d80cb1 2019-02-19 17:10:49.360310
0d45103a 2019-02-19 17:10:45.459008
481a04b2 2019-02-19 17:10:38.249354
